In [1]:
from queue import PriorityQueue, LifoQueue
import numpy as np
import random as rnd

In [5]:
class Cell: 
    OPEN = 0
    BLOCKED = -1
    ON_FIRE = 2
    def __init__(self, row, col, prev = None):
        self.row = row
        self.col = col
        self.prev = prev
    def __lt__(self, other):        
        if (self.row == other.row):
            return self.col < other.col
        return self.row < other.row
    def __eq__(self, other):
        return self.row == other.row and self.col == other.col and self.prev is other.prev
    def __str__(self):
        return "(" + str(self.row) + ", " + str(self.col) + ") :: prev=" + ("NONE" if self.prev is None else "(" + str(self.prev.row) + ", " + str(self.prev.col) + ")")
    def __repr__(self):
        return "(" + str(self.row) + ", " + str(self.col) + ") :: prev=" + ("NONE" if self.prev is None else "(" + str(self.prev.row) + ", " + str(self.prev.col) + ")")

In [6]:
class Maze:
    def __init__(self, dim, p, q = 0):
        self.board =  np.zeros([dim, dim], dtype=int)
        self.dim = dim
        self.currentfire = []
        self.q = q
        self.p = p
        obstaclesList = []
        for i in range(0, dim): #range() generates a list from 0 to dim-1 which can take up a lot of memory
            for j in range(0, dim):             
                if(rnd.random() < p):
                    self.board[i, j] = Cell.BLOCKED
                    obstaclesList.append((i, j))
        self.obstacles = np.array(obstaclesList)
        self.board[0, 0] = Cell.OPEN
        self.board[dim - 1, dim - 1] = Cell.OPEN

    def isSolvable(self):
        return pathExists(self)
    
    def startFire(self):
        i = rnd.randint(0,self.dim-1)
        j = rnd.randint(0,self.dim-1)
        while(self.board[i,j] == -1 or (i==0 and j == 0) or (i == self.dim-1 and j == self.dim-1)):
            i = rnd.randint(0,self.dim-1)
            j = rnd.randint(0,self.dim-1)
        self.board[i,j] = Cell.ON_FIRE
        self.currentfire.append([i,j,None])
    
    def iterateFire(self):
        visited = [] #neighbors already visited in current iteration
        nowonfire = [] #need to append all new fires at the end 
        for f in self.currentfire:
            if(f[2] != 0): #check to see if all neighbors are already on fire. 
                neighbors = findNeighboringFlammableCoords(f[0:2],self) #calculate valid neighbors of f, the cell that is currently on fire
                ofnbor = 0 #counter to find out how many neighbors are on fire
                for n in neighbors: #loop through the neighbors of f
                    nr,nc = n 
                    if (self.board[nr,nc] == Cell.ON_FIRE): 
                        ofnbor = ofnbor + 1 
                    else:
                        if(n[0:2] not in visited): #only roll once for each cell not on fire
                            if(self.board[nr,nc] != Cell.BLOCKED):
                                surroundings = findNeighboringFlammableCoords(n[0:2],self) #see what the neighbors are for the neighbor of f
                                k = 0 #k will be guaranteed to be at least 1
                                for nn in surroundings:
                                    nnr,nnc = nn
                                    if(self.board[nnr,nnc] == Cell.ON_FIRE):
                                        k = k + 1 #counter
                                if(rnd.random() <= (1-((1-self.q)**k))): #roll
                                    self.board[nr,nc] = Cell.ON_FIRE 
                                    nowonfire.append([nr,nc,None])
                                    ofnbor = ofnbor + 1
                                else:
                                    visited.append([nr,nc])
                if(ofnbor == len(neighbors)):
                        f[2] = 0
        if(nowonfire):
            for x in nowonfire:
                self.currentfire.append(x)

In [7]:
def pathExists(maze, fromCoords = (0,0), toCoords = None):
    if toCoords is None:
        toCoords = (maze.dim - 1, maze.dim - 1)
    toRow, toCol = toCoords
    visited = set()
    fringe = LifoQueue()
    fringe.put(fromCoords)
    while not fringe.empty():
        currentCoords = fringe.get()
        currentRow, currentCol = currentCoords
        if (currentRow == toRow and currentCol == toCol):
            return True
        else:
            neighbors = findNeighboringOpenCoords(currentCoords, maze)
            for neighbor in neighbors:
                if neighbor in visited:
                    continue
                fringe.put(neighbor)
            visited.add(currentCoords)
    return False

In [8]:
def findNeighboringOpenCoords(coords, maze):
    cellRow, cellCol = coords
    potentialNeighbors = [(cellRow + 1, cellCol), (cellRow - 1, cellCol), (cellRow, cellCol - 1), (cellRow, cellCol + 1)]
    neighbors = []
    for potentialNeighbor in potentialNeighbors:
        row, col = potentialNeighbor
        if (row >= maze.dim or row < 0 or col >= maze.dim or col < 0 or maze.board[row,col] != Cell.OPEN):
            continue
        neighbors.append(potentialNeighbor)
    return neighbors

def findNeighboringOpenCells(cell, maze):
    neighboringCoords = findNeighboringOpenCoords((cell.row, cell.col), maze)
    neighbors = []
    for neighbor in neighboringCoords:
        row, col = neighbor
        neighbors.append(Cell(row, col, cell))
    return neighbors

def findNeighboringFlammableCoords(coords,maze): 
    cellRow, cellCol = coords
    potentialNeighbors = [(cellRow + 1, cellCol), (cellRow - 1, cellCol), (cellRow, cellCol - 1), (cellRow, cellCol + 1)]
    neighbors = []
    for potentialNeighbor in potentialNeighbors:
        row, col = potentialNeighbor
        if (row >= maze.dim or row < 0 or col >= maze.dim or col < 0 or maze.board[row,col] == Cell.BLOCKED):
            continue
        neighbors.append(potentialNeighbor)
    return neighbors

In [9]:
def uniformCost(maze, cell):
    return 1
def shortestPathSearch(maze, startCoords = (0, 0), heuristicFunction = uniformCost, findNeighbors = findNeighboringOpenCells):
    startRow, startCol = startCoords
    startCell = Cell(startRow, startCol)
    visited = set()
    fringe = PriorityQueue()
    fringe.put((0, startCell))
    while not fringe.empty():
        pathLength, currentCell = fringe.get()
        if (currentCell.row == maze.dim - 1 and currentCell.col == maze.dim - 1):
            shortestPath = []
            while (currentCell != startCell):
                shortestPath.append((currentCell.row, currentCell.col))
                currentCell = currentCell.prev
            shortestPath.append((startRow, startCol))
            shortestPath.reverse()
            return shortestPath
        else:
            neighbors = findNeighbors(cell = currentCell, maze = maze)
            for neighbor in neighbors:
                if (neighbor.row, neighbor.col) in visited:
                    continue
                nextPathLength = pathLength + heuristicFunction(maze = maze, cell = neighbor)
                fringe.put((nextPathLength, neighbor))
            visited.add((currentCell.row, currentCell.col))
    return None

In [8]:
m1 = Maze(15, .3, 0.5)
while (m1.isSolvable() is False):
    m1 = Maze(15,0.3,0.5)
print(m1.board)

[[ 0  0 -1  0  0  0  0 -1  0  0 -1 -1  0  0 -1]
 [-1  0  0  0 -1  0  0 -1  0 -1 -1 -1  0 -1  0]
 [-1  0  0  0 -1  0 -1  0 -1 -1 -1  0  0  0 -1]
 [ 0  0 -1  0 -1  0  0  0  0  0  0 -1 -1  0  0]
 [ 0  0  0 -1  0  0  0  0 -1  0 -1  0 -1  0  0]
 [ 0  0 -1  0  0  0  0 -1  0 -1  0  0  0 -1  0]
 [-1  0  0  0  0  0  0  0 -1 -1  0 -1 -1  0  0]
 [ 0 -1  0 -1 -1  0  0  0  0 -1  0 -1  0 -1  0]
 [ 0  0 -1 -1 -1 -1 -1  0  0  0 -1  0  0  0  0]
 [ 0 -1  0  0  0  0  0  0  0 -1  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0  0 -1  0 -1  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0 -1  0 -1 -1  0 -1  0]
 [-1 -1  0  0  0 -1  0  0  0 -1  0 -1  0  0 -1]
 [ 0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0]
 [ 0 -1  0  0  0  0  0  0 -1 -1 -1 -1  0  0  0]]


In [9]:
m1.startFire()
print(m1.board)

[[ 0  0 -1  0  0  0  0 -1  0  0 -1 -1  0  0 -1]
 [-1  0  0  0 -1  0  0 -1  0 -1 -1 -1  0 -1  0]
 [-1  0  0  0 -1  0 -1  0 -1 -1 -1  0  0  0 -1]
 [ 0  0 -1  0 -1  0  0  0  0  0  0 -1 -1  0  0]
 [ 0  0  0 -1  0  0  0  0 -1  0 -1  0 -1  0  0]
 [ 0  0 -1  0  0  0  0 -1  0 -1  0  0  0 -1  0]
 [-1  0  0  0  0  0  0  0 -1 -1  0 -1 -1  0  0]
 [ 0 -1  0 -1 -1  0  0  0  0 -1  0 -1  0 -1  0]
 [ 0  0 -1 -1 -1 -1 -1  0  0  0 -1  0  0  0  0]
 [ 0 -1  0  0  0  0  0  0  0 -1  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0  0 -1  0 -1  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0 -1  0 -1 -1  0 -1  0]
 [-1 -1  0  0  0 -1  0  0  0 -1  0 -1  0  0 -1]
 [ 0  2  0  0  0 -1  0  0  0  0  0  0  0  0  0]
 [ 0 -1  0  0  0  0  0  0 -1 -1 -1 -1  0  0  0]]


In [34]:
m1.iterateFire()
print(m1.board)

[[ 0  0 -1  0  0  0  0 -1  0  0 -1 -1  0  0 -1]
 [-1  0  0  0 -1  0  0 -1  0 -1 -1 -1  0 -1  0]
 [-1  0  0  0 -1  0 -1  0 -1 -1 -1  0  0  0 -1]
 [ 0  0 -1  0 -1  0  0  0  0  0  0 -1 -1  0  0]
 [ 0  0  0 -1  0  0  0  0 -1  0 -1  0 -1  0  0]
 [ 0  0 -1  0  0  0  0 -1  0 -1  0  0  0 -1  0]
 [-1  0  0  0  0  2  2  2 -1 -1  0 -1 -1  0  0]
 [ 2 -1  0 -1 -1  2  2  2  2 -1  0 -1  0 -1  0]
 [ 2  2 -1 -1 -1 -1 -1  2  2  2 -1  0  0  0  0]
 [ 2 -1  2  2  2  2  2  2  2 -1  0  0  0  0  0]
 [ 2  2 -1  2  2  2  2  2 -1  0 -1  0  0  0  0]
 [ 2  2  2  2 -1  2  2  2 -1  0 -1 -1  0 -1  0]
 [-1 -1  2  2  2 -1  2  2  2 -1  2 -1  2  2 -1]
 [ 2  2  2  2  2 -1  2  2  2  2  2  2  2  2  2]
 [ 2 -1  2  2  2  2  2  2 -1 -1 -1 -1  2  2  2]]


In [35]:
print(m1.currentfire)

[[13, 1, 0], [13, 2, 0], [13, 0, 0], [14, 2, 0], [13, 3, 0], [14, 0, 0], [14, 3, 0], [12, 3, 0], [13, 4, 0], [12, 2, 0], [14, 4, 0], [12, 4, 0], [11, 2, 0], [14, 5, 0], [11, 3, 0], [11, 1, 0], [14, 6, 0], [13, 6, 0], [10, 3, 0], [12, 6, 0], [13, 7, 0], [10, 1, 0], [10, 4, 0], [11, 6, 0], [12, 7, 0], [11, 0, 0], [14, 7, 0], [10, 0, 0], [9, 4, 0], [10, 5, 0], [9, 3, 0], [13, 8, 0], [10, 6, 0], [11, 5, 0], [11, 7, 0], [12, 8, 0], [9, 0, 0], [9, 5, 0], [13, 9, 0], [10, 7, 0], [9, 6, 0], [8, 0, 0], [13, 10, 0], [9, 7, 0], [7, 0, 0], [12, 10, 0], [8, 7, 0], [9, 2, 0], [8, 8, 0], [9, 8, 0], [7, 7, 0], [8, 1, 0], [13, 11, 0], [7, 8, 0], [8, 9, 0], [13, 12, None], [6, 7, 0], [12, 12, None], [13, 13, 0], [7, 6, 0], [6, 6, None], [14, 13, 0], [12, 13, 0], [13, 14, None], [7, 5, None], [6, 5, None], [14, 12, None], [14, 14, None]]
